In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import math
import numpy as np

In [2]:
x = Variable(torch.Tensor([[1,2,3],[4,5,6]]))
y = Variable(torch.Tensor([[7,8],[9,10],[11,12]]))

In [3]:
torch.dot(x,y)

Variable containing:
 217
[torch.FloatTensor of size 1]

In [4]:
torch.randn(4,3)


-1.4562 -0.6410  0.6894
 0.6798 -1.1414 -0.3796
-0.0938 -0.4423 -1.1183
 0.5069 -0.3467  0.3102
[torch.FloatTensor of size 4x3]

In [5]:
Variable(torch.randn(4,3))

Variable containing:
 0.2523  1.1193 -0.5316
-0.8734 -0.4813 -0.1955
 0.6738 -1.6651  0.2236
 1.4731 -0.7909  0.8869
[torch.FloatTensor of size 4x3]

In [5]:
def mf(R, K, alpha=0.0002, beta=0.02, step=10000):
    P = Variable(torch.randn(K,R.size(0)))
    Q = Variable(torch.randn(K,R.size(1)))
    e_ui = 0
    errs = []
    for s in range(step):
        for u in range(R.size(0)):
            for i in range(R.size(1)):
                if R[u,i].data[0] == 0:
                    continue
                e_ui = R[u,i] - torch.dot(P[:,u],Q[:,i])
                for k in range(K):
                    P[k,u] = P[k, u] + alpha * (2 * e_ui * Q[k,i] - beta * P[k,u])
                    Q[k,i] = Q[k, i] + alpha * (2 * e_ui * P[k,u] - beta * Q[k,i])
        error_R = torch.matmul(torch.t(P),Q)
        e = 0
        for u in range(R.size(0)):
            for i in range(R.size(1)):
                if R[u,i].data[0] == 0:
                    continue
                e = e + torch.pow(R[u,i] - torch.dot(P[:,u],Q[:,i]),2)
                for k in range(K):
                    e =  e + 0.5 * beta * torch.pow(P[k][u],2) + torch.pow(Q[k][i],2)
        errs.append(e.data[0])
        if s % 100 == 0 :
            print(e.data[0])
    return P,Q, errs
    

In [6]:
R = Variable(torch.FloatTensor([
        [5, 3, 0, 1],
        [4, 0, 0, 1],
        [1, 1, 0, 5],
        [1, 0, 0, 4],
        [0, 1, 5, 4],
        ]
    ))

In [7]:
#mf(R, 2)

In [9]:
a = Variable(torch.Tensor(5,6).uniform_(-1,1))
relu = torch.nn.ReLU
print(a)
print(relu(a))
# print(aa)

Variable containing:
 0.7875  0.1924  0.2367  0.2955  0.9673  0.1931
 0.8374 -0.5782  0.9000  0.7750 -0.5620  0.4693
-0.9157  0.7180  0.7439  0.5742  0.9180  0.1809
-0.2145 -0.8647 -0.9344  0.5862  0.1713 -0.9328
 0.4236 -0.5280  0.9849  0.8556 -0.8292 -0.2644
[torch.FloatTensor of size 5x6]



RuntimeError: bool value of Variable objects containing non-empty torch.FloatTensor is ambiguous

In [44]:
class MF(nn.Module):
    def __init__(self, input_size, K): 
        super(MF,self).__init__() #最初の呪文
        self.input_size = input_size
        self.P = nn.Parameter(torch.Tensor(K, input_size[0]).uniform_(0,1)) #パラメーターを指定
        self.Q = nn.Parameter(torch.Tensor(K, input_size[1]).uniform_(0,1))
        
    def non_negative(self,K):
        for k in range(K):
            for i in range(R.size[0]):
                if self.P[k,i] <= 0:
                    self.P = 0
            for j in range(R.size[1]):
                if self.Q[k,j] <= 0:
                    self.Q = 0
                    
    def forward(self, R=None): #Outputをだすメソッド
        if R is not None: #引数としてRがあれば，0を考慮しましょう．
            weight = (R != 0).float() #floatにすることで，True,Falseを1,0という数字に変換している．
        else: #引数としてRがなければ，0は考慮しない．
            weight = Variable(torch.ones(self.input_size)) #0を考慮しない場合，weightを1の正方行列にすることで，下のreturnを変化させないようにしている．
        return torch.matmul(torch.t(self.P), self.Q) * weight #weightをかけて0だったら消して，1だったらそのままにする．

def ll_gaussian(R, mean):
    a = -0.5 * R.size(0) * R.size(1) * math.log(2 * math.pi)
    b = -0.5 * torch.sum(torch.pow(x - mean,2))
    return a + b

def mll_poisson(R, pq):
    I = Variable(torch.ones(R.size(0), R.size(1)))
    return torch.sum(-R * (torch.log(pq) * I) + pq)

def map_pred(p, q, lam):
    return math.log(2 * math.pi / lam) + lam / 2 * (torch.norm(p, 2) + torch.norm(q, 2))

In [46]:
model = MF((R.size(0), R.size(1)), 2)
optimizer = optim.Adam(model.parameters(), lr = 0.002) #更新方法をAdam，更新するものはmodel.parametersで，learning rateを0.002で指定．
creterion = nn.MSELoss(size_average=False) #nn.ModuleでMSELossを算出するもの
for i in range(100):
    optimizer.zero_grad() # 勾配の初期化
    pred = model(R) #forwardが実行される
#    loss = torch.sum(torch.pow(R - pred, 2))
    loss = mll_poisson(R, pred) + map_pred(model.P, model.Q, 0.002) #Lossの合計を計算
    a = mll_poisson(R,pred)
    b = map_pred(model.P, model.Q, 0.2)
    loss.backward() #微分
    optimizer.step() #Lossをそれぞれで微分したものを引く作業
    model.non_negative(2)
#    model.P = torch.nn.functional.relu(model.P)
#    model.Q = torch.nn.functional.relu(model.Q)

    print(a,b)
    print(loss.data[0]) #Lossがいくつだったか表示

TypeError: 'builtin_function_or_method' object is not subscriptable

In [ ]:
R

In [ ]:
for i in range(R.size(0)):
    for j in range(R.size(1)):
        if R[i,j].data[0] <= 1:
            continue
        R[i,j] = torch.prod(torch.range(1,R[i,j].data[0]))

In [ ]:
torch.rand(3,4)

In [ ]:
torch.prod(torch.range(1,R[i,j].data[0]))

In [ ]:
I = Variable(torch.ones(R.size(0), R.size(1)))

In [ ]:
I

In [ ]:
R * I

In [ ]:
model()

In [ ]:
print(model.P, model.Q)

In [ ]:
model.P.data